In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ChatBot Deva/nexyug_chatbot_dataset_3000.xlsx')

In [30]:
df.count()

,0
User Input,53
Bot Response,65
Intent,53


In [31]:
df.isnull().sum()

,0
User Input,12
Bot Response,0
Intent,12


In [32]:
df.head()

,User Input,Bot Response,Intent
0,What is your name?,Hello! I’m Deva. How can I help you today?,name
1,Hi,Hello! I’m Deva. How can I help you today?,greeting
2,Hello,Hi there! Deva at your service. What can I do ...,greeting
3,Hey,Hey! Deva here. How can I assist you?,greeting
4,How are you?,"I’m good, thank you! How about you?",small_talk


In [33]:
df = df.dropna(subset=['User Input', 'Intent'])

In [34]:
df.isnull().sum()

,0
User Input,0
Bot Response,0
Intent,0


In [35]:
df["User Input"]=df["User Input"].str.lower()
df["Bot Response"]=df["Bot Response"].str.lower()
df["Intent"]=df["Intent"].str.lower()
df.head(2)

,User Input,Bot Response,Intent
0,what is your name?,hello! i’m deva. how can i help you today?,name
1,hi,hello! i’m deva. how can i help you today?,greeting


In [36]:
pattern_to_remove = r'#\d+'
df["User Input"]=df["User Input"].str.replace(pattern_to_remove, '', regex=True)
df["User Input"]=df["User Input"].str.replace(r'[^\w\s]', '', regex=True)
df.head(2)

,User Input,Bot Response,Intent
0,what is your name,hello! i’m deva. how can i help you today?,name
1,hi,hello! i’m deva. how can i help you today?,greeting


In [37]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [38]:
df['User Input'] = df['User Input'].fillna('').astype(str)
df['tokenized_user_input'] = df['User Input'].apply(word_tokenize)

In [39]:
df.head()

,User Input,Bot Response,Intent,tokenized_user_input
0,what is your name,hello! i’m deva. how can i help you today?,name,"[what, is, your, name]"
1,hi,hello! i’m deva. how can i help you today?,greeting,[hi]
2,hello,hi there! deva at your service. what can i do ...,greeting,[hello]
3,hey,hey! deva here. how can i assist you?,greeting,[hey]
4,how are you,"i’m good, thank you! how about you?",small_talk,"[how, are, you]"


In [40]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords_from_list(tokens):
    return [word for word in tokens if word not in stop_words]

df['filtered_user_input'] = df['tokenized_user_input'].apply(remove_stopwords_from_list)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,User Input,Bot Response,Intent,tokenized_user_input,filtered_user_input
0,what is your name,hello! i’m deva. how can i help you today?,name,"[what, is, your, name]",[name]
1,hi,hello! i’m deva. how can i help you today?,greeting,[hi],[hi]
2,hello,hi there! deva at your service. what can i do ...,greeting,[hello],[hello]
3,hey,hey! deva here. how can i assist you?,greeting,[hey],[hey]
4,how are you,"i’m good, thank you! how about you?",small_talk,"[how, are, you]",[]


In [41]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['encoded_intent'] = label_encoder.fit_transform(df['Intent'])
df[['Intent', 'encoded_intent']].tail()

,Intent,encoded_intent
60,legal_info,14
61,legal_info,14
62,contact_info,8
63,service_info,19
64,legal_info,14


In [42]:
# !pip install transformers torch

In [43]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [44]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [45]:
# def get_transformer_embeddings(text, tokenizer, model, max_length=128):
#     inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     inputs = {key: val.to(device) for key, val in inputs.items()}
#     model.to(device)

#     with torch.no_grad():
#       outputs = model(**inputs)
#     sentence_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

#     return sentence_embedding

In [46]:
# df['text_for_embedding'] = df['filtered_user_input'].apply(lambda x: ' '.join(x))

# if 'text_for_embedding' not in df.columns:
#     print("Error: 'text_for_embedding' column not found. Please ensure you re-joined your filtered tokens first.")
#     print("Run: df['text_for_embedding'] = df['filtered_user_input'].apply(lambda x: ' '.join(x))")
# else:
#     df['user_input_embeddings'] = df['text_for_embedding'].apply(
#         lambda x: get_transformer_embeddings(x, tokenizer, model)
#     )
#     if not df['user_input_embeddings'].empty:
#         print(f"\nShape of the first embedding: {df['user_input_embeddings'].iloc[0].shape}")
#     else:
#         print("\n'user_input_embeddings' column is empty after generation.")
# df[['filtered_user_input',"text_for_embedding","user_input_embeddings"]].tail()

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Step 1: Vectorize user input
vectorizer = TfidfVectorizer()
df['joined_input'] = df['filtered_user_input'].apply(lambda x: ' '.join(x))
X = vectorizer.fit_transform(df['joined_input'])
y = df['Intent']

# Step 2: Train intent classifier
clf = LogisticRegression()
clf.fit(X, y)

LogisticRegression()

In [ ]:
known_intents = df['Intent'].unique().tolist()

def get_response(user_input):
    # Clean and normalize input
    user_input_lower = user_input.strip().lower()

    # Match input exactly to known intents
    if user_input_lower in [intent.lower() for intent in known_intents]:
        # Choose a random bot response for this intent
        response = df[df['Intent'].str.lower() == user_input_lower]['Bot Response'].sample(1).values[0]
        return response
    else:
        return "Sorry, I didn't understand that. Please try asking in a different way."

print(get_response("CEO"))

hello! i’m deva. how can i help you today?


In [49]:
print(get_response("name"))

service_info
certainly! here's the link to our mobile development services: https://quantum-dev-xi.vercel.app/services/mobile-development


In [50]:
get_response("owner of your company")

website_request


"sure. here's the website link: https://quantum-dev-xi.vercel.app/"

In [51]:
import joblib

joblib.dump(clf, 'intent_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')


['vectorizer.pkl']

In [52]:
import joblib
clf = joblib.load('intent_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

In [53]:
user_input = "name"
X_input = vectorizer.transform([user_input])
predicted_intent = clf.predict(X_input)
print("Predicted Intent:", predicted_intent[0])

Predicted Intent: service_info
